In [9]:
import sys
import librosa
import os
from os import path
from os import listdir
from os.path import isfile, join
import numpy as np
import scipy.io as spio
import scipy.io.wavfile as sciwav
from tqdm import tqdm
np.set_printoptions(threshold=sys.maxsize)

In [10]:
gspeechcmd_full_wav_file = "../../../../Speech_Corpora/google_speech_command_dataset/speech_commands_v0.01/bird/00b01445_nohash_0.wav"
nctu_wav_test_file = "../../../../MyWork/NCTU/DNN_VAD/data/clean speech/MIR-1K/ABJones_1_lyrics.wav"

In [11]:
# def test_read_wav(wavfilename):
#     wav_ary, sr = librosa.load(nctu_wav_test_file)
#     wav_16000 = librosa.resample(wav_ary,sr,16000)
#     len_wav_16000 = len(wav_16000)
    
#     print("length of wav_ary is {}, sample rate is {}".format(len(wav_ary),sr))
#     print("length of wav_16000 is {}, sample rate is {}".format(len(wav_16000),16000))
#     return wav_16000, len_wav

# Quantization of wav read using librosa.load
This sounds like a quantization problem. If samples in the wave file are stored as float and librosa is just performing a straight cast to an int, and value less than 1 will be truncated to 0. More than likely, this is why sig is an array of all zeros. The float must be scaled to map it into range of an int. For example,

>>> a = sp.randn(10)
>>> a
array([-0.04250369,  0.244113  ,  0.64479281, -0.3665814 , -0.2836227 ,
       -0.27808428, -0.07668698, -1.3104602 ,  0.95253315, -0.56778205])
Convert a to type int without scaling

>>> a.astype(int)
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Convert a to int with scaling for 16-bit integer

>>> b = (a* 32767).astype(int)
>>> b
array([ -1392,   7998,  21127, -12011,  -9293,  -9111,  -2512, -42939,
        31211, -18604])
Convert scaled int back to float

>>> c = b/32767.0
>>> c
array([-0.04248177,  0.24408704,  0.64476455, -0.36655782, -0.28360851,
       -0.27805414, -0.0766625 , -1.31043428,  0.9525132 , -0.56776635])
c and b are only equal to about 3 or 4 decimal places due to quantization to int.

If librosa is returning a float, you can scale it by 2**15 and cast it to an int to get same range of values that scipy wave reader is returning. Since librosa is returning a float, chances are the values going to lie within a much smaller range, such as [-1, +1], than a 16-bit integer which will be in [-32768, +32767]. So you need to scale one to get the ranges to match. For example,

sig, rate = librosa.load(spec_file, mono=True)
sig = sig × 32767

In [7]:
def read_and_convert_wav_to_16K_SR(wav_file):
    std_sr = 16000
#     raw_wav, sr = librosa.load(wav_file)
    sr, sig = wavio.read(wav_file)
    print("sample rate is {}".format(sr))
    return sig

In [8]:
# test_read_wav(nctu_wav_test_file)
wav_ = read_and_convert_wav_to_16K_SR(nctu_wav_test_file)
with open("./wav_out_test_2.txt","w") as fw:
    fw.write(str(wav_))
    fw.flush()

sample rate is 16000


## divide audio signal into frames
If you know for sure that your signal is 2 seconds long, then the sampling frequency can be found from dt*N=2, where dt=1/fs is sampling period and N is number of samples, which you don't need.
IF you already have an array of sampled signal, it has particular length (number of samples). In addition, you know that the 'duration' of that array is 2 seconds, so, you must break your array onto 2000ms/100ms=20 pieces:

In [13]:
""" Matlab codes
x=[your array of N samples];
n=round(length(x)/20); %find how many samples will each frame contain
P=zeros(n,20); %preallocate the matrix for 20 colums of Nsamples/20 in each
for k=0:19
    P(:,k+1)=x(1+n*k:n*(k+1));
end
"""

' Matlab codes\nx=[your array of N samples]; n=round(length(x)/20); %find how many samples will each frame contain P=zeros(n,20); %preallocate the matrix for 20 colums of Nsamples/20 in each for k=0:19     P(:,k+1)=x(1+n*k:n*(k+1)); end '

In [ ]:
def getFilesInFloder(folderPath):
    onlyfiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f))]
    return onlyfiles

In [ ]:
def convert2mat(filename):
    """mat file description:
    each row contains 40 elements
    each row is 100ms
    1 frame is 25ms, beacuase 
    """
    pass

In [ ]:
# if __name__ == "__main__":
#     flag = path.exists(full_wav_file)
#     if flag:
#         test_read_wav(full_wav_file)
#     else:
#         print("file doesn't exist.")

In [8]:
# test_read_one_frame_and_plot(full_wav_file)

In [6]:
# import matplotlib.pyplot as plt
# import matplotlib.pylab as plt

# def plot(vector, xlabel=None, ylabel=None):
#     plt.figure()
#     plt.plot(vector)
#     plt.xlabel(xlabel)
#     plt.ylabel(ylabel)
#     plt.plot()


# def test_read_one_frame_and_plot(wavfilename):
#     wav2_ary, sr2 = librosa.load(wavfilename)
#     wav2_16000 = librosa.resample(wav2_ary,sr2,16000)
#     plot(wav2_16000,"whole")
#     start_point=1600
#     step_quantity=1600
#     wav_len=16000
#     for idx in range(start_point,wav_len,step_quantity):
# #         idx1 = idx-1600
# #         idx2 = idx-1200
# #         idx3 = idx-800
# #         idx4 = idx-400
#         _s=idx-1600
#         _e=idx
#         print("the {}-th segment values are {}".format(idx/1600,wav2_16000[_s:_e]))
#         plot(wav2_16000[_s:_e],idx/1600)
#         print("*******************************************************************")